<a href="https://colab.research.google.com/github/acmilannesta/Bert-embedding/blob/master/GoogleQA_Lindada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow-gpu==2.1.0-rc0

     |████████████████████████████████| 402.3MB 36kB/s 
     |████████████████████████████████| 450kB 49.5MB/s 
     |████████████████████████████████| 3.8MB 62.5MB/s 
     |████████████████████████████████| 81kB 11.6MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.2 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.10.0 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled goo

In [0]:
!nvidia-smi

Wed Dec 25 02:41:06 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import tensorflow as tf
tf.__version__

'1.15.0'

In [0]:
!top

=top - 03:42:46 up  1:07,  0 users,  load average: 0.16, 0.42, 0.46
Tasks:  14 total,   1 running,  12 sleeping,   0 stopped,   1 zombie
%Cpu(s):  9.2 us,  4.6 sy,  0.0 ni, 85.1 id,  1.1 wa,  0.0 hi,  0.0 si,  0.0 st
KiB Mem : 26753328 total,  4413976 free,  5600876 used, 16738476 buff/cache
KiB Swap:        0 total,        0 free,        0 used. 23877696 avail Mem 

    PID USER      PR  NI    VIRT    RES    SHR S  %CPU %MEM     TIME+ COMMAND   
      1 root      20   0   39196   6372   4848 S   0.0  0.0   0:00.04 run.sh    
      8 root      20   0  706192  71420  24976 S   0.0  0.3   0:08.81 node      
     24 root      20   0  488056 109556  26348 S   0.0  0.4   0:10.23 jupyter-+ 
    138 root      20   0   35888   4892   3796 S   0.0  0.0   0:00.13 tail      
    247 root      20   0   18376   1492   1192 S   0.0  0.0   0:00.00 bash      
    248 root      20   0 1123684  15584  13676 S   0.0  0.1   0:00.01 drive     
    249 root      20   0   11596   2124   1848 S   0.0  0.0   

In [0]:
!kill 3454

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
os.chdir("/content/gdrive/My Drive/GoogleQA/code/")

In [0]:
!ls

bert_lstm.ipynb  bert_weights_RepeatKfold  tf_bert.ipynb     tf_bert_v3.ipynb
bert_v4.ipynb	 lstm_use.ipynb		   tf_bert_v2.ipynb


In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, GroupKFold, KFold, RepeatedKFold
from tqdm import tqdm
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow.keras.backend as K
import gc
import re
import os
import sys
import six
import collections
from scipy.stats import spearmanr, rankdata
from math import floor, ceil

sys.path.extend(['../input/bert-joint-baseline/'])
# from transformers import *
import tokenization
DEBUG = False
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# print('tokenizer: ', tokenizer)

if not DEBUG:
    os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = "true"
    np.set_printoptions(suppress=True)

PATH = '../input/google-quest-challenge/'
BERT_PATH = '../input/bert-en-uncased-l12-h768-a12'
# BERT_PATH = '../input/bertenuncasedl24h1024a16'
tokenizer = tokenization.FullTokenizer(BERT_PATH+'/assets/vocab.txt', False)
MAX_SEQUENCE_LENGTH = 512

if DEBUG:
    df_train = pd.read_csv(PATH+'train.csv', nrows=30)
    df_test = pd.read_csv(PATH+'test.csv', nrows=30)
else:
    df_train = pd.read_csv(PATH + 'train.csv')
    df_test = pd.read_csv(PATH + 'test.csv')
print('train shape =', df_train.shape)
print('test shape =', df_test.shape)

output_categories = list(df_train.columns[11:])
print('\noutput categories:\n\t', output_categories)

targets = [
        'question_asker_intent_understanding',
        'question_body_critical',
        'question_conversational',
        'question_expect_short_answer',
        'question_fact_seeking',
        'question_has_commonly_accepted_answer',
        'question_interestingness_others',
        'question_interestingness_self',
        'question_multi_intent',
        'question_not_really_a_question',
        'question_opinion_seeking',
        'question_type_choice',
        'question_type_compare',
        'question_type_consequence',
        'question_type_definition',
        'question_type_entity',
        'question_type_instructions',
        'question_type_procedure',
        'question_type_reason_explanation',
        'question_type_spelling',
        'question_well_written',
        'answer_helpful',
        'answer_level_of_information',
        'answer_plausible',
        'answer_relevance',
        'answer_satisfaction',
        'answer_type_instructions',
        'answer_type_procedure',
        'answer_type_reason_explanation',
        'answer_well_written'
    ]

train shape = (6079, 41)
test shape = (476, 11)

output categories:
	 ['question_asker_intent_understanding', 'question_body_critical', 'question_conversational', 'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer', 'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice', 'question_type_compare', 'question_type_consequence', 'question_type_definition', 'question_type_entity', 'question_type_instructions', 'question_type_procedure', 'question_type_reason_explanation', 'question_type_spelling', 'question_well_written', 'answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance', 'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 'answer_type_reason_explanation', 'answer_well_written']


In [0]:
def _get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens) > max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1] * len(tokens) + [0] * (max_seq_length - len(tokens))


def _get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens) > max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))


def _get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length - len(token_ids))
    return input_ids

def my_pad(text, max_length, tokenizer):
    res = tokenizer.tokenize(text)
    if len(res) > max_length:
        head_length = int(0.25 * max_length)
        tail_length = max_length - head_length
        res = res[:head_length] + res[-tail_length:]
    return res

def _trim_input(title, question, answer, max_sequence_length,
                t_max_len=30, q_max_len=239, a_max_len=239):
    t = tokenizer.tokenize(title)
    q = tokenizer.tokenize(question)
    a = tokenizer.tokenize(answer)

    t_len = len(t)
    q_len = len(q)
    a_len = len(a)

    if (t_len + q_len + a_len + 4) > max_sequence_length:

        if t_max_len > t_len:
            t_new_len = t_len
            a_max_len = a_max_len + floor((t_max_len - t_len) / 2)
            q_max_len = q_max_len + ceil((t_max_len - t_len) / 2)
        else:
            t_new_len = t_max_len

        if a_max_len > a_len:
            a_new_len = a_len
            q_new_len = q_max_len + (a_max_len - a_len)
        elif q_max_len > q_len:
            a_new_len = a_max_len + (q_max_len - q_len)
            q_new_len = q_len
        else:
            a_new_len = a_max_len
            q_new_len = q_max_len

        if t_new_len + a_new_len + q_new_len + 4 != max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d"
                             % (max_sequence_length, (t_new_len + a_new_len + q_new_len + 4)))

        head_t_new_len = int(0.25 * t_new_len)
        tail_t_new_len = t_new_len - head_t_new_len

        head_q_new_len = int(0.25 * q_new_len)
        tail_q_new_len = q_new_len - head_q_new_len

        head_a_new_len = int(0.25 * a_new_len)
        tail_a_new_len = a_new_len - head_a_new_len

        t = t[:head_t_new_len] + t[-tail_t_new_len:]
        q = q[:head_q_new_len] + q[-tail_q_new_len:]
        a = a[:head_a_new_len] + a[-tail_a_new_len:]

    return t, q, a

def prepare_data(df, tokenizer):
    input_ids, input_masks, input_segments = [], [], []
    for _, instance in tqdm(df[['question_title', 'question_body', 'answer', 'question_user_name', 'answer_user_name']].iterrows()):
        question_title_ = str(instance['question_title']).lower()
        question_body_ = str(instance['question_body']).lower()
        answer_ = str(instance['answer']).lower()
#         question_user_name_ = instance['question_user_name']
#         answer_user_name_ = instance['answer_user_name']

#         question_title_ = my_pad(question_title_, max_length=15, tokenizer=tokenizer)
#         question_body_ = my_pad(question_body_, max_length=245, tokenizer=tokenizer)
#         answer_ = my_pad(answer_, max_length=248, tokenizer=tokenizer)
#         question_user_name_ = my_pad(question_user_name_, max_length=2, tokenizer=tokenizer)
#         answer_user_name_ = my_pad(answer_user_name_, max_length=2, tokenizer=tokenizer)

        question_title_, question_body_, answer_ = _trim_input(question_title_, question_body_, answer_, MAX_SEQUENCE_LENGTH,
                                                               t_max_len=15, q_max_len=245, a_max_len=248)

        stoken = ["[CLS]"] + question_title_ + [","] + question_body_ + ["[SEP]"] + answer_ + ["[SEP]"]

        input_ids_ = _get_ids(stoken, tokenizer, MAX_SEQUENCE_LENGTH)
        input_masks_ = _get_masks(stoken, MAX_SEQUENCE_LENGTH)
        input_segments_ = _get_segments(stoken, MAX_SEQUENCE_LENGTH)

        input_ids.append(input_ids_)
        input_masks.append(input_masks_)
        input_segments.append(input_segments_)

    return [np.asarray(input_ids, dtype=np.int32),
            np.asarray(input_masks, dtype=np.int32),
            np.asarray(input_segments, dtype=np.int32)]

def compute_output_arrays(df, columns):
    return np.asarray(df[columns])

outputs = compute_output_arrays(df_train, output_categories)

inputs = prepare_data(df_train, tokenizer)
test_inputs = prepare_data(df_test, tokenizer)

6079it [00:24, 249.87it/s]
476it [00:01, 242.73it/s]


In [0]:
class MyLabelEncoder(object):
    """safely handle unknown label"""
    def __init__(self):
        self.mapper = {}

    def fit(self, X):
        uniq_X = np.unique(X)
        # reserve 0 for unknown
        self.mapper = dict(zip(uniq_X, range(1, len(uniq_X) + 1)))
        return self

    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)

    def _map(self, x):
        return self.mapper.get(x, 0)

    def transform(self, X):
        return list(map(self._map, X))
def get_cate_feat(df, isTrain):
    if isTrain:
        label_encoder["category"] = MyLabelEncoder()
        category_feat = np.array(label_encoder["category"].fit_transform(df["category"]))
        label_encoder["host"] = MyLabelEncoder()
        host_feat = np.array(label_encoder["host"].fit_transform(df["host"]))
    else:
        category_feat = np.array(label_encoder["category"].transform(df["category"]))
        host_feat = np.array(label_encoder["host"].transform(df["host"]))
    return [category_feat, host_feat]
label_encoder = {}
cate_feat_tr = get_cate_feat(df_train, isTrain=True)
cate_feat_te = get_cate_feat(df_test, isTrain=False)

inputs.extend(cate_feat_tr)
test_inputs.extend(cate_feat_te)

In [0]:
label_encoder = {}
label_encoder["category"] = MyLabelEncoder()
label_encoder["category"].fit(df_train["category"])
label_encoder["host"] = MyLabelEncoder()
label_encoder["host"].fit(df_test["host"])

def data_generator(data, batch_size, shuffle=False, isTrain=True):
  idx = np.arange(len(data))
  if shuffle:
    np.random.shuffle(idx)
  batches = [idx[range(batch_size*i, min(len(data), batch_size*(i+1)))] for i in range(len(data)/batch_size+1)]
  while True:
    for i in batches:
      data_slice = data.iloc[i]
      if isTrain:
        y = data_slice[targets].values

      category_feat = np.array(label_encoder["category"].transform(data_slice["category"]))
      host_feat = np.array(label_encoder["host"].transform(data_slice["host"]))

      input_ids, input_masks, input_segments = [], [], []
      for _, instance in tqdm(data_slice[['question_title', 'question_body', 'answer']].iterrows()):
        question_title_ = str(instance['question_title']).lower()
        question_body_ = str(instance['question_body']).lower()
        answer_ = str(instance['answer']).lower()
        
        question_title_, question_body_, answer_ = _trim_input(question_title_, question_body_, answer_, MAX_SEQUENCE_LENGTH,
                                                                t_max_len=15, q_max_len=245, a_max_len=248)
        
        stoken = ["[CLS]"] + question_title_ + [","] + question_body_ + ["[SEP]"] + answer_ + ["[SEP]"]
        
        input_ids_ = _get_ids(stoken, tokenizer, MAX_SEQUENCE_LENGTH)
        input_masks_ = _get_masks(stoken, MAX_SEQUENCE_LENGTH)
        input_segments_ = _get_segments(stoken, MAX_SEQUENCE_LENGTH)
        
        input_ids.append(input_ids_)
        input_masks.append(input_masks_)
        input_segments.append(input_segments_)
      X = [np.asarray(input_ids, dtype=np.int32), np.asarray(input_masks, dtype=np.int32), np.asarray(input_segments, dtype=np.int32),
           category_feat, host_feat]

      if isTrain:
        yield (X, y)
      else:
        yield X

In [0]:
# https://www.kaggle.com/hireme/fun-api-keras-f1-metric-cyclical-learning-rate/code
class CyclicLR(tf.keras.callbacks.Callback):
    """This callback implements a cyclical learning rate policy (CLR).
    The method cycles the learning rate between two boundaries with
    some constant frequency, as detailed in this paper (https://arxiv.org/abs/1506.01186).
    The amplitude of the cycle can be scaled on a per-iteration or
    per-cycle basis.
    This class has three built-in policies, as put forth in the paper.
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "triangular2":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exp_range":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each
        cycle iteration.
    For more detail, please see paper.

    # Example
        ```python
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., mode='triangular')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```

    Class also supports custom scaling functions:
        ```python
            clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., scale_fn=clr_fn,
                                scale_mode='cycle')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```
    # Arguments
        base_lr: initial learning rate which is the
            lower boundary in the cycle.
        max_lr: upper boundary in the cycle. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore
            max_lr may not actually be reached depending on
            scaling function.
        step_size: number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch.
        mode: one of {triangular, triangular2, exp_range}.
            Default 'triangular'.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
        gamma: constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
        scale_fn: Custom scaling policy defined by a single
            argument lambda function, where
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored
        scale_mode: {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on
            cycle number or cycle iterations (training
            iterations since start of cycle). Default is 'cycle'.
    """

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1 / (2. ** (x - 1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma ** (x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.

    def clr(self):
        cycle = np.floor(1 + self.clr_iterations / (2 * self.step_size))
        x = np.abs(self.clr_iterations / self.step_size - 2 * cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr - self.base_lr) * np.maximum(0, (1 - x)) * self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr - self.base_lr) * np.maximum(0, (1 - x)) * self.scale_fn(
                self.clr_iterations)

    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())

    def on_batch_end(self, epoch, logs=None):

        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

        K.set_value(self.model.optimizer.lr, self.clr())

In [0]:
import tensorflow as tf
from tensorflow.python.keras.optimizer_v2.optimizer_v2 import OptimizerV2
from tensorflow.python import ops, math_ops, state_ops, control_flow_ops
from tensorflow.python.keras import backend_config

__all__ = ['AdamWarmup']


class AdamWarmup(OptimizerV2):
    """Adam optimizer with warmup."""

    def __init__(self,
                 decay_steps,
                 warmup_steps,
                 min_lr=0.0,
                 learning_rate=0.001,
                 beta_1=0.9,
                 beta_2=0.999,
                 epsilon=1e-7,
                 weight_decay=0.,
                 weight_decay_pattern=None,
                 amsgrad=False,
                 name='Adam',
                 **kwargs):
        r"""Construct a new Adam optimizer.

        Args:
            decay_steps: Learning rate will decay linearly to zero in decay steps.
            warmup_steps: Learning rate will increase linearly to lr in first warmup steps.
            lr: float >= 0. Learning rate.
            beta_1: float, 0 < beta < 1. Generally close to 1.
            beta_2: float, 0 < beta < 1. Generally close to 1.
            epsilon: float >= 0. Fuzz factor. If `None`, defaults to `K.epsilon()`.
            weight_decay: float >= 0. Weight decay.
            weight_decay_pattern: A list of strings. The substring of weight names to be decayed.
                                  All weights will be decayed if it is None.
            amsgrad: boolean. Whether to apply the AMSGrad variant of this
                algorithm from the paper "On the Convergence of Adam and
                Beyond".
        """

        super(AdamWarmup, self).__init__(name, **kwargs)
        self._set_hyper('decay_steps', float(decay_steps))
        self._set_hyper('warmup_steps', float(warmup_steps))
        self._set_hyper('min_lr', min_lr)
        self._set_hyper('learning_rate', kwargs.get('lr', learning_rate))
        self._set_hyper('decay', self._initial_decay)
        self._set_hyper('beta_1', beta_1)
        self._set_hyper('beta_2', beta_2)
        self._set_hyper('weight_decay', weight_decay)
        self.epsilon = epsilon or backend_config.epsilon()
        self.amsgrad = amsgrad
        self._initial_weight_decay = weight_decay
        self._weight_decay_pattern = weight_decay_pattern

    def _create_slots(self, var_list):
        for var in var_list:
            self.add_slot(var, 'm')
        for var in var_list:
            self.add_slot(var, 'v')
        if self.amsgrad:
            for var in var_list:
                self.add_slot(var, 'vhat')

    def set_weights(self, weights):
        params = self.weights
        num_vars = int((len(params) - 1) / 2)
        if len(weights) == 3 * num_vars + 1:
            weights = weights[:len(params)]
        super(AdamWarmup, self).set_weights(weights)

    def _resource_apply_dense(self, grad, var):
        var_dtype = var.dtype.base_dtype
        lr_t = self._decayed_lr(var_dtype)
        m = self.get_slot(var, 'm')
        v = self.get_slot(var, 'v')
        beta_1_t = self._get_hyper('beta_1', var_dtype)
        beta_2_t = self._get_hyper('beta_2', var_dtype)
        epsilon_t = ops.convert_to_tensor(self.epsilon, var_dtype)
        local_step = math_ops.cast(self.iterations + 1, var_dtype)
        beta_1_power = math_ops.pow(beta_1_t, local_step)
        beta_2_power = math_ops.pow(beta_2_t, local_step)

        decay_steps = self._get_hyper('decay_steps', var_dtype)
        warmup_steps = self._get_hyper('warmup_steps', var_dtype)
        min_lr = self._get_hyper('min_lr', var_dtype)
        lr_t = tf.where(
            local_step <= warmup_steps,
            lr_t * (local_step / warmup_steps),
            min_lr + (lr_t - min_lr) * (1.0 - tf.minimum(local_step, decay_steps) / decay_steps),
        )
        lr_t = (lr_t * math_ops.sqrt(1 - beta_2_power) / (1 - beta_1_power))

        m_t = state_ops.assign(m,
                               beta_1_t * m + (1.0 - beta_1_t) * grad,
                               use_locking=self._use_locking)

        v_t = state_ops.assign(v,
                               beta_2_t * v + (1.0 - beta_2_t) * math_ops.square(grad),
                               use_locking=self._use_locking)

        if self.amsgrad:
            v_hat = self.get_slot(var, 'vhat')
            v_hat_t = math_ops.maximum(v_hat, v_t)
            var_update = m_t / (math_ops.sqrt(v_hat_t) + epsilon_t)
        else:
            var_update = m_t / (math_ops.sqrt(v_t) + epsilon_t)

        if self._initial_weight_decay > 0.0:
            weight_decay = self._get_hyper('weight_decay', var_dtype)
            var_update += weight_decay * var
        var_update = state_ops.assign_sub(var, lr_t * var_update, use_locking=self._use_locking)

        updates = [var_update, m_t, v_t]
        if self.amsgrad:
            updates.append(v_hat_t)
        return control_flow_ops.group(*updates)

    def _resource_apply_sparse(self, grad, var, indices):
        var_dtype = var.dtype.base_dtype
        lr_t = self._decayed_lr(var_dtype)
        beta_1_t = self._get_hyper('beta_1', var_dtype)
        beta_2_t = self._get_hyper('beta_2', var_dtype)
        epsilon_t = ops.convert_to_tensor(self.epsilon, var_dtype)
        local_step = math_ops.cast(self.iterations + 1, var_dtype)
        beta_1_power = math_ops.pow(beta_1_t, local_step)
        beta_2_power = math_ops.pow(beta_2_t, local_step)

        decay_steps = self._get_hyper('decay_steps', var_dtype)
        warmup_steps = self._get_hyper('warmup_steps', var_dtype)
        min_lr = self._get_hyper('min_lr', var_dtype)
        lr_t = tf.where(
            local_step <= warmup_steps,
            lr_t * (local_step / warmup_steps),
            min_lr + (lr_t - min_lr) * (1.0 - tf.minimum(local_step, decay_steps) / decay_steps),
        )
        lr_t = (lr_t * math_ops.sqrt(1 - beta_2_power) / (1 - beta_1_power))

        m = self.get_slot(var, 'm')
        m_scaled_g_values = grad * (1 - beta_1_t)
        m_t = state_ops.assign(m, m * beta_1_t, use_locking=self._use_locking)
        with ops.control_dependencies([m_t]):
            m_t = self._resource_scatter_add(m, indices, m_scaled_g_values)

        v = self.get_slot(var, 'v')
        v_scaled_g_values = (grad * grad) * (1 - beta_2_t)
        v_t = state_ops.assign(v, v * beta_2_t, use_locking=self._use_locking)
        with ops.control_dependencies([v_t]):
            v_t = self._resource_scatter_add(v, indices, v_scaled_g_values)

        if self.amsgrad:
            v_hat = self.get_slot(var, 'vhat')
            v_hat_t = math_ops.maximum(v_hat, v_t)
            var_update = m_t / (math_ops.sqrt(v_hat_t) + epsilon_t)
        else:
            var_update = m_t / (math_ops.sqrt(v_t) + epsilon_t)

        if self._initial_weight_decay > 0.0:
            weight_decay = self._get_hyper('weight_decay', var_dtype)
            var_update += weight_decay * var
        var_update = state_ops.assign_sub(var, lr_t * var_update, use_locking=self._use_locking)

        updates = [var_update, m_t, v_t]
        if self.amsgrad:
            updates.append(v_hat_t)
        return control_flow_ops.group(*updates)

    def get_config(self):
        config = super(AdamWarmup, self).get_config()
        config.update({
            'decay_steps': self._serialize_hyperparameter('decay_steps'),
            'warmup_steps': self._serialize_hyperparameter('warmup_steps'),
            'min_lr': self._serialize_hyperparameter('min_lr'),
            'learning_rate': self._serialize_hyperparameter('learning_rate'),
            'decay': self._serialize_hyperparameter('decay'),
            'beta_1': self._serialize_hyperparameter('beta_1'),
            'beta_2': self._serialize_hyperparameter('beta_2'),
            'weight_decay': self._serialize_hyperparameter('weight_decay'),
            'epsilon': self.epsilon,
            'amsgrad': self.amsgrad,
        })
        return config

def calc_train_steps(num_example, batch_size, epochs, warmup_proportion=0.1):
    """Calculate the number of total and warmup steps.
    (320, 32)
    :param num_example: Number of examples in one epoch.
    :param batch_size: Batch size.
    :param epochs: Number of epochs.
    :param warmup_proportion: The proportion of warmup steps.
    :return: Total steps and warmup steps.
    """
    steps = (num_example + batch_size - 1) // batch_size
    total = steps * epochs
    warmup = int(total * warmup_proportion)
    return total, warmup

In [0]:
class SpearmanRhoCallback(tf.keras.callbacks.Callback):
    def __init__(self, training_data, validation_data, patience, model_name):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]

        self.patience = patience
        self.value = -1
        self.bad_epochs = 0
        self.model_name = model_name

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred_val = self.model.predict(self.x_val)
        rho_val = np.mean([spearmanr(self.y_val[:, ind],
                                     y_pred_val[:, ind]).correlation
                           for ind in range(y_pred_val.shape[1])])
        if rho_val >= self.value:
            self.value = rho_val
            self.bad_epochs = 0
            self.model.save_weights(self.model_name)
        else:
            self.bad_epochs += 1
        if self.bad_epochs >= self.patience:
            print("Epoch %05d: early stopping Threshold" % epoch)
            self.model.stop_training = True
        print('\rval_spearman-rho: %s' % (str(round(rho_val, 4))), end=100 * ' ' + '\n')
        logs['val_rho'] = rho_val
        return rho_val

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return
import itertools

def spearman_loss(y_true, y_pred):
  return K.var(y_pred - y_true, axis=-1) / (K.std(y_pred, axis=-1)*K.std(y_true, axis=-1))# +K.random_uniform(shape=y_true.shape, minval=0.0, maxval=0.0001))

def bert_model(bert_trainabel, learning_rate, len_tr, BATCH_SIZE, NUM_EPOCHS):
    EMB_SIZE = 32

    input_word_ids = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    input_segments = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_segments')

    input_category = tf.keras.layers.Input((1,), dtype=tf.int32, name='input_category')
    input_host = tf.keras.layers.Input((1,), dtype=tf.int32, name='input_host')

    category_emb = tf.keras.layers.SpatialDropout1D(0.1)(
        tf.keras.layers.Embedding(input_dim=6, output_dim=EMB_SIZE)(input_category))
    host_emb = tf.keras.layers.SpatialDropout1D(0.1)(
        tf.keras.layers.Embedding(input_dim=65, output_dim=EMB_SIZE)(input_host))
    features_dense = tf.keras.layers.concatenate([category_emb, host_emb], axis=1)
    features_dense = tf.keras.layers.Flatten()(features_dense)

    bert_layer = hub.KerasLayer(BERT_PATH, trainable=bert_trainabel)
    pooled_output, sequence_output = bert_layer([input_word_ids, input_masks, input_segments])

    print('pooled_output: ', pooled_output)
    print('sequence_output: ', sequence_output)

    # bert_feat = tf.keras.layers.Dense(EMB_SIZE, activation="relu")(pooled_output)
    # fm_emb = [bert_feat, category_emb, host_emb]
    # sum_add = tf.keras.layers.add(fm_emb)
    # sum_add = tf.keras.layers.multiply([sum_add,sum_add])
    # add_sum = []
    # for layer in fm_emb:
    #     add_sum.append(tf.keras.layers.multiply([layer,layer]))
    # add_sum = tf.keras.layers.add(add_sum)
        
    # subtract_layer = tf.keras.layers.Lambda(lambda inputs: inputs[0] - inputs[1],output_shape=lambda shapes: shapes[0])
    # fm_part = subtract_layer([sum_add, add_sum])
    # fm_part  = tf.keras.layers.Lambda(lambda x: x * 0.5)(fm_part)
    # fm_part = tf.keras.layers.Dropout(0.5)(fm_part)
    # fm_part = tf.keras.layers.Flatten()(fm_part)
    # print(fm_part)

    x = tf.keras.layers.GlobalAveragePooling1D()(sequence_output)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.concatenate([x, features_dense])

    out = tf.keras.layers.Dense(30, activation="sigmoid", name="dense_output")(x)

    model = tf.keras.models.Model(inputs=[input_word_ids, input_masks, input_segments, input_category, input_host],
                                  outputs=out)
    decay_steps, warmup_steps = calc_train_steps(
        len_tr,
        batch_size=BATCH_SIZE,
        epochs=NUM_EPOCHS,
        )
    adamW_opt = AdamWarmup(decay_steps=decay_steps, warmup_steps=warmup_steps, lr=learning_rate, min_lr=0,)
    # Nadam = tf.keras.optimizers.Nadam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
    model.compile(loss='binary_crossentropy', optimizer=adamW_opt)

    return model

In [0]:
import time
# folds = GroupKFold(n_splits=5).split(X=df_train.question_title, groups=df_train.question_title)
# folds = KFold(n_splits=3, shuffle=True, random_state=2019).split(outputs)
kf = RepeatedKFold(n_splits=5, n_repeats=2, random_state=0)
idx = [x for x in kf.split(df_train, groups=df_train.question_title)]

y_test = np.zeros((len(test_inputs), len(targets)))
all_predictions = []
score = []
score2 = []
score3 = []
start_time = time.time()
model_path = 'bert_weights_RepeatKfold'
if not os.path.exists(model_path):
    os.mkdir(model_path)
# for fold_, (trn_idx, val_idx) in enumerate(folds):
for fold_, (trn_idx, val_idx) in enumerate(idx[:2], 9):
  print('fold_: ', fold_)
#   if fold_<4:
#     continue
  K.clear_session()
  
  X_train = [inputs[i][trn_idx] for i in range(len(inputs))]
  X_valid = [inputs[i][val_idx] for i in range(len(inputs))]
  y_train = outputs[trn_idx]
  y_valid = outputs[val_idx]

  data_train = df_train.iloc[trn_idx]
  data_valid = df_train.iloc[val_idx]
  
  model_weight = f'{model_path}/bert_fold{fold_}.h5'
  model = bert_model(bert_trainabel=True, learning_rate=5e-5, len_tr=len(data_train), BATCH_SIZE=4, NUM_EPOCHS=4)
  model.fit(X_train, y_train, epochs=4, batch_size=4,
              callbacks=[SpearmanRhoCallback(training_data=(X_train, y_train), validation_data=(X_valid, y_valid),
                                             patience=1, model_name=model_weight, ),
                        # CyclicLR(base_lr=2e-5, max_lr=3e-5, step_size=300., mode='exp_range', gamma=0.99994)
                        ])
  
  model = bert_model(bert_trainabel=False, learning_rate=1e-3, len_tr=len(data_train), BATCH_SIZE=32, NUM_EPOCHS=4)
  model.load_weights(model_weight)
  model.fit(X_train, y_train, epochs=4, batch_size=32,
              callbacks=[SpearmanRhoCallback(training_data=(X_train, y_train), validation_data=(X_valid, y_valid),
                                             patience=3, model_name=model_weight,),
                        # CyclicLR(base_lr=1e-3, max_lr=2e-3, step_size=300., mode='exp_range', gamma=0.99994)
                        ])
  model.load_weights(model_weight)
  
  all_predictions.append(model.predict(test_inputs))
  
  valid_pred = model.predict(X_valid)
  slice_score = np.mean([spearmanr(y_valid[:, ind], valid_pred[:, ind]).correlation
                           for ind in range(valid_pred.shape[1])])
  print('fold{} score: {}'.format(fold_, slice_score))

  # valid_pred2 = np.round(valid_pred, 2)
  # slice_score2 = np.mean([spearmanr(y_valid[:, ind], valid_pred2[:, ind]).correlation
  #                          for ind in range(valid_pred2.shape[1])])
  # print('fold{} score2: {}'.format(fold_, slice_score2))

  # valid_pred3 = (valid_pred//(1/90))*(1/90)
  # slice_score3 = np.mean([spearmanr(y_valid[:, ind], valid_pred3[:, ind]).correlation
  #                          for ind in range(valid_pred3.shape[1])])
  # print('fold{} score3: {}'.format(fold_, slice_score3))

  score.append(slice_score)
  # score2.append(slice_score2)
  # score3.append(slice_score3)
  print("time elapsed: {:<5.2}m".format((time.time() - start_time) / 60))
print('CV score: {}'.format(np.mean(score)))
# print('CV score: {}'.format(np.mean(score2)))
# print('CV score: {}'.format(np.mean(score3)))

fold_:  9
pooled_output:  Tensor("keras_layer/Identity:0", shape=(None, 768), dtype=float32)
sequence_output:  Tensor("keras_layer/Identity_1:0", shape=(None, None, 768), dtype=float32)
Train on 4863 samples
Epoch 1/4
val_spearman-rho: 0.3712                                                                                                    
4863/4863 [==============================] - 439s 90ms/sample - loss: 0.4115
Epoch 2/4
val_spearman-rho: 0.4019                                                                                                    
4863/4863 [==============================] - 407s 84ms/sample - loss: 0.3655
Epoch 3/4
val_spearman-rho: 0.4122                                                                                                    
4863/4863 [==============================] - 406s 84ms/sample - loss: 0.3411
Epoch 4/4
4860/4863 [============================>.] - ETA: 0s - loss: 0.3144Epoch 00003: early stopping Threshold
val_spearman-rho: 0.4078                 

In [0]:
fold0 score: 0.39553310346725007
fold1 score: 0.4024302061634733
fold2 score: 0.3911486329250454
fold3 score: 0.38787471016487524
fold4 score: 0.3958322311099684

In [0]:
CV score: 0.39443920279732525

In [0]:
(0.4028767893921821+0.4054496422760184+0.39370800478615+0.40809677256587584+0.39268607409936+0.39282118200606153+0.4053617546791114+0.40862098614678144+0.4157536902010747+0.40483598938929527) / 10

0.40302108855419105

In [0]:
(0.4028767893921821+0.4054496422760184+0.39370800478615+0.4157536902010747+0.40483598938929527) / 5

0.4045248232089441

In [0]:
fold1 score: 0.4028767893921821
fold2 score: 0.4054496422760184
fold3 score: 0.39370800478615
fold4 score: 0.40809677256587584
fold5 score: 0.39268607409936
fold6 score: 0.39282118200606153
fold7 score: 0.4053617546791114
fold8 score: 0.40862098614678144
fold9 score: 0.4157536902010747
fold10 score: 0.40483598938929527
CV score: 0.40302108855419105